In [5]:
import pandas as pd
import requests
import json
from pprint import pprint

In [2]:
#base url
base_url = 'https://fantasy.premierleague.com/api/'

r = requests.get(base_url +'bootstrap-static/').json()

In [9]:
# show the top level fields
pprint(r, indent=2, depth=1, compact=True)

{ 'element_stats': [...],
  'element_types': [...],
  'elements': [...],
  'events': [...],
  'game_settings': {...},
  'phases': [...],
  'teams': [...],
  'total_players': 10997846}


# Player data

In [11]:
players = r['elements']

pprint((players[0]))

{'assists': 0,
 'bonus': 0,
 'bps': 5,
 'chance_of_playing_next_round': 100,
 'chance_of_playing_this_round': 100,
 'clean_sheets': 0,
 'clean_sheets_per_90': 0.0,
 'code': 58822,
 'corners_and_indirect_freekicks_order': None,
 'corners_and_indirect_freekicks_text': '',
 'cost_change_event': 0,
 'cost_change_event_fall': 0,
 'cost_change_start': -3,
 'cost_change_start_fall': 3,
 'creativity': '1.6',
 'creativity_rank': 395,
 'creativity_rank_type': 143,
 'direct_freekicks_order': None,
 'direct_freekicks_text': '',
 'dreamteam_count': 0,
 'element_type': 2,
 'ep_next': '0.5',
 'ep_this': '0.5',
 'event_points': 0,
 'expected_assists': '0.00979',
 'expected_assists_per_90': 0.03147,
 'expected_goal_involvements': '0.00979',
 'expected_goal_involvements_per_90': 0.03147,
 'expected_goals': '0.00000',
 'expected_goals_conceded': '0.04350',
 'expected_goals_conceded_per_90': 0.13982,
 'expected_goals_per_90': 0.0,
 'first_name': 'Cédric',
 'form': '0.0',
 'form_rank': 386,
 'form_rank_typ